Before we start, we need to make sure that we have a Kafka cluster running and a topic that produces some streaming data. For simplicity, we will use a single-node Kafka cluster and a topic named `users`. Open the `4.0 user-gen-kafka.ipynb` notebook and execute the cell. This notebook produces a user record every few seconds and put it on a Kafka topic called users. 

In [11]:
from delta import  # Delta Lake 라이브러리 임포트 configure_spark_with_delta_pip
from pyspark.sql import SparkSession  # Spark SQL 작업을 위한 SparkSession 임포트
from pyspark.sql.functions import  # Spark SQL 함수들 임포트 col, from_json, avg
from pyspark.sql.types import  # Spark SQL 데이터 타입 임포트 StructType, StructField, IntegerType, StringType

In [12]:
builder = (SparkSession.builder  # SparkSession 빌더 패턴 시작
           .appName("transform-filter-streaming")  # 애플리케이션 이름 설정
           .master("spark://spark-master:7077")  # Spark 마스터 URL 설정
           .config("spark.executor.memory", "512m")  # Spark 설정 옵션
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")  # Spark 설정 옵션
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")  # Spark 설정 옵션)

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()  # SparkSession 생성 또는 기존 세션 반환
spark.sparkContext.setLogLevel("ERROR")  # 로그 레벨을 ERROR로 설정

In [13]:
df = (spark.readStream  # 스트리밍 데이터 읽기
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "users")
      .option("startingOffsets", "earliest")
      .load(  # 파일 로드))

In [14]:
schema = StructType  # 구조체 타입([
    StructField  # 구조체 필드('id', IntegerType(), True),
    StructField  # 구조체 필드('name', StringType(), True),
    StructField  # 구조체 필드('age', IntegerType(), True),
    StructField  # 구조체 필드('gender', StringType(), True),
    StructField  # 구조체 필드('country', StringType(), True)])

df = df.withColumn(  # 새 컬럼 추가 또는 기존 컬럼 수정'value', from_json(col('value').cast("STRING"), schema))

In [15]:
df = df.select(  # 컬럼 선택
    col(  # 컬럼 참조'value.id').alias('id'),
    col(  # 컬럼 참조'value.name').alias('name'),
    col(  # 컬럼 참조'value.age').alias('age'),
    col(  # 컬럼 참조'value.gender').alias('gender'),
    col(  # 컬럼 참조'value.country').alias('country'))

In [16]:
df = (df.select(  # 컬럼 선택'age','country', 'gender').filter("age >= 21").groupBy('country', 'gender').agg(avg('age').alias('avg_age')))

In [17]:
query = (df.writeStream  # 스트리밍 데이터 쓰기
    .outputMode(  # 스트리밍 출력 모드 설정'complete')
    .format('console')
    .start()  # 스트리밍 시작)

-------------------------------------------
Batch: 0
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|              59.5|
|Australia|     F|              43.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|             44.25|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|              59.5|
|Australia|     F|              43.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|              59.5|
|Australia|     F|              40.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              40.0|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|              45.0|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 5
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|              31.5|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 6
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|              50.0|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 7
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|             50.75|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              32.0|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 8
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|             50.75|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              28.5|
|    India|     F|50.666666666666664|
|       UK|     F|             34.25|
+---------+------+------------------+



-------------------------------------------
Batch: 9
-------------------------------------------
+---------+------+------------------+
|  country|gender|           avg_age|
+---------+------+------------------+
|   Brazil|     F|38.666666666666664|
|   Brazil|     M|              34.0|
|      USA|     F|60.666666666666664|
|Australia|     F|              36.6|
|   Canada|     M|              43.0|
|       UK|     M|              22.5|
|      USA|     M|             50.75|
|    India|     M|45.666666666666664|
|    China|     M|41.333333333333336|
|    China|     F| 41.42857142857143|
|   Canada|     F|              31.5|
|Australia|     M|              28.5|
|    India|     F|50.666666666666664|
|       UK|     F|              32.6|
+---------+------+------------------+



In [18]:
query.stop()

24/02/04 17:54:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=true]] is aborting.
24/02/04 17:54:32 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=true]] aborted.


In [20]:
spark.stop()  # Spark 세션 종료 - 리소스 정리 